In [ ]:
import pandas as pd
import numpy as np
import random
import sys
import os
!pip install tldextract -q
import tldextract
import warnings
import regex as re
import eli5
from typing import *

from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

import matplotlib.pyplot as plt
import seaborn as sns 
from urllib.parse import urlparse
from nltk.tokenize import RegexpTokenizer

warnings.filterwarnings("ignore")

In [ ]:
url_data = pd.read_csv('/kaggle/input/phishing-site-urls/phishing_site_urls.csv')
#pd.set_option('display.max_colwidth', -1)
url_data.sample(10)

# Introduction

In this notebook I go over the preprocessing of URLs and compare three different binary classification models for determining if a URL is legitimate or used in a phishing attempt. I also look at what features are most highly weighted in the most accurate model. As this is the first notebook I have created, I would greatly appreciate any feedback you may have.

A thank you to [Matthew Franglen](https://www.kaggle.com/matthewfranglen) for improving the quality of my code!

# Preprocessing

In [ ]:
url_data = url_data.rename(columns={"URL": "url", "Label": "label"})

To extract useful features from the URL, we first will break it down into useful parts using the urlparse method. This returns a dictionary of values of the url which will be added as columns in the dataframe soon.

In [ ]:
def parse_url(url: str) -> Optional[Dict[str, str]]:
    try:
        no_scheme = not url.startswith('https://') and not url.startswith('http://')
        if no_scheme:
            parsed_url = urlparse(f"http://{url}")
            return {
                "scheme": None, # not established a value for this
                "netloc": parsed_url.netloc,
                "path": parsed_url.path,
                "params": parsed_url.params,
                "query": parsed_url.query,
                "fragment": parsed_url.fragment,
            }
        else:
            parsed_url = urlparse(url)
            return {
                "scheme": parsed_url.scheme,
                "netloc": parsed_url.netloc,
                "path": parsed_url.path,
                "params": parsed_url.params,
                "query": parsed_url.query,
                "fragment": parsed_url.fragment,
            }
    except:
        return None

In [ ]:
url_data["parsed_url"] = url_data.url.apply(parse_url)
url_data

In [ ]:
url_data = pd.concat([
    url_data.drop(['parsed_url'], axis=1),
    url_data['parsed_url'].apply(pd.Series)
], axis=1)
url_data

An initial problem faced was that urlparse would not parse some unreadable URLs. The following removes any column where the netloc has no value (ie when the parse_url method returns None).

In [ ]:
url_data = url_data[~url_data.netloc.isnull()]
url_data

The first meaningful bit of data to extract is the length of the URL.

In [ ]:
url_data["length"] = url_data.url.str.len()

The TLD is then extracted using a python library, and if no TLD is present simply add 'None'.

In [ ]:
url_data["tld"] = url_data.netloc.apply(lambda nl: tldextract.extract(nl).suffix)
url_data['tld'] = url_data['tld'].replace('','None')

Next is a regex to determine if the URL is an IP address.

In [ ]:
url_data["is_ip"] = url_data.netloc.str.fullmatch(r"\d+\.\d+\.\d+\.\d+")

The next few sections relate to certain punctuation in the URL which may be an indicator one way or another that a URL is malicious. My reasoning behind this is that typosquatted domains (which are almost always malicious) may contain this punctation to appear similar to a legitimate domain. There may also be more of each in the path of the URL for a legitimate URL as blogs often use underscores in a URL. 

In [ ]:
url_data['domain_hyphens'] = url_data.netloc.str.count('-')
url_data['domain_underscores'] = url_data.netloc.str.count('_')
url_data['path_hyphens'] = url_data.path.str.count('-')
url_data['path_underscores'] = url_data.path.str.count('_')
url_data['slashes'] = url_data.path.str.count('/')

Full stops in the path could indicate that theres an attempt to fool a user into thinking a domain is legit. For example, attacker.com/paypal.com may be used to trick a user. Full stops may also be a sign of files in the URL such as shell.exe

In [ ]:
url_data['full_stops'] = url_data.path.str.count('.')

Similar to the previous datapoint, getting the full stops in a subdomain will count how many subdomains are present. Lots may be another visual trick such as paypal.com.attacker.com/

In [ ]:
def get_num_subdomains(netloc: str) -> int:
    subdomain = tldextract.extract(netloc).subdomain 
    if subdomain == "":
        return 0
    return subdomain.count('.') + 1

url_data['num_subdomains'] = url_data['netloc'].apply(lambda net: get_num_subdomains(net))

As previous notebooks have shown, the lexical features of the URL will be important. In this instance, I have decided to separate the tokens from the path and the domain itself. My thinking here is that the same word in a path and domain may have very different meanings. By this i mean if you see 'paypal' in a URL path, it may be a malicious URL which is trying to seem legitimate, but 'paypal' in the domain may be more legitimate. 

In [ ]:
tokenizer = RegexpTokenizer(r'[A-Za-z]+')
def tokenize_domain(netloc: str) -> str:
    split_domain = tldextract.extract(netloc)
    no_tld = str(split_domain.subdomain +'.'+ split_domain.domain)
    return " ".join(map(str,tokenizer.tokenize(no_tld)))
         
url_data['domain_tokens'] = url_data['netloc'].apply(lambda net: tokenize_domain(net))

In [ ]:
url_data['path_tokens'] = url_data['path'].apply(lambda path: " ".join(map(str,tokenizer.tokenize(path))))

The labels are now extracted and the URL column removed.

In [ ]:
url_data_y = url_data['label']
url_data.drop('label', axis=1, inplace=True)
url_data.drop('url', axis=1, inplace=True)
url_data.drop('scheme', axis=1, inplace=True)
url_data.drop('netloc', axis=1, inplace=True)
url_data.drop('path', axis=1, inplace=True)
url_data.drop('params', axis=1, inplace=True)
url_data.drop('query', axis=1, inplace=True)
url_data.drop('fragment', axis=1, inplace=True)
url_data

# Training

When using pipelines and vectorizers, you need a converter to feed the vectorizer every word of that column. It cannot add the values one row at a time and so a converter class must be created.

In [ ]:
class Converter(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, data_frame):
        return data_frame.values.ravel()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(url_data, url_data_y, test_size=0.2)

The numeric features need their own pipeline to scale the data, MinMaxScaler was used as MultinomialNB needs no negative values to work.

In [ ]:
numeric_features = ['length', 'domain_hyphens', 'domain_underscores', 'path_hyphens', 'path_underscores', 'slashes', 'full_stops', 'num_subdomains']
numeric_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler())])

The only categorical feature is TLD and OneHot encoding will be used for this. Interestingly there is no difference between using this or converting and using the TfidfVectorizer. However, using OneHot encoding makes the TLD obvious in the feature importance section.

In [ ]:
categorical_features = ['tld', 'is_ip']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

CountVectorizer and TfidfVectorizer produce very similar results, but with the best performing model (spoiler its SVC) Tfidf slightly improved the score.

In [ ]:
vectorizer_features = ['domain_tokens','path_tokens']
vectorizer_transformer = Pipeline(steps=[
    ('con', Converter()),
    ('tf', TfidfVectorizer())])

The next step is to link all the transformers together in a ColumnTransformer, and create a pipeline for each classifier.

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('domvec', vectorizer_transformer, ['domain_tokens']),
        ('pathvec', vectorizer_transformer, ['path_tokens'])
    ])

svc_clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LinearSVC())])

log_clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression())])

nb_clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', MultinomialNB())])

The models can then be fit on the training data.

In [ ]:
svc_clf.fit(X_train, y_train)
log_clf.fit(X_train, y_train)
nb_clf.fit(X_train, y_train)

# Results

To display the results I created a method that prints the score, classification report and creates a heat map for each model. 

In [ ]:
def results(name: str, model: BaseEstimator) -> None:
    preds = model.predict(X_test)

    print(name + " score: %.3f" % model.score(X_test, y_test))
    print(classification_report(y_test, preds))
    labels = ['Good', 'Bad']

    conf_matrix = confusion_matrix(y_test, preds)

    font = {'family' : 'normal',
            'size'   : 14}

    plt.rc('font', **font)
    plt.figure(figsize= (10,6))
    sns.heatmap(conf_matrix, xticklabels=labels, yticklabels=labels, annot=True, fmt="d", cmap='Blues')
    plt.title("Confusion Matrix for " + name)
    plt.ylabel('True Class')
    plt.xlabel('Predicted Class')

In [ ]:
results("SVC" , svc_clf)
results("Logistic Regression" , log_clf)
results("Naive Bayes" , nb_clf)

As we see the SVC performs best out of the three followed by MultinomialNB. While Logistic Regression performs the worst, we can see it produces less false negatives than Naive Bayes. 

Also if numerical features are removed, logistic regression performs better. I wouldn't know why this would be the case and would be interested to hear some ideas for it.

# Feature Importance

Finally, to see what features are most strongly weighted to the SVC classifier I use eli5 to show this. It is worth noting that weights may be high for rarer features and should be taken with a grain of salt.

In [ ]:
onehot_columns = list(svc_clf.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot'].get_feature_names(input_features=categorical_features))
domvect_columns = list(svc_clf.named_steps['preprocessor'].named_transformers_['domvec'].named_steps['tf'].get_feature_names())
pathvect_columns = list(svc_clf.named_steps['preprocessor'].named_transformers_['pathvec'].named_steps['tf'].get_feature_names())
numeric_features_list = list(numeric_features)
numeric_features_list.extend(onehot_columns)
numeric_features_list.extend(domvect_columns)
numeric_features_list.extend(pathvect_columns)
eli5.explain_weights(svc_clf.named_steps['classifier'], top=20, feature_names=numeric_features_list)